# Novozyme Enzyme Stability Prediction

This notebook contains model training and evaluation to predict the thermal stability (as measured via melting point) of enzymes based on their amino acid sequence.

Competition details are available [here](https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/overview).

Prepared for SCS3546 - Deep Learning

<pre> Christopher Eeles </pre>

<pre> X361483 </pre>

Please note that the dependencies for this notebook are available in a Conda
environment file on GitHub under `ChristopherEeles/enzyme_thermal_stability_prediction/env`

## Dataset Download

Retrieve the dataset from Kaggle via the Kaggle API utility

In [1]:
from pathlib import Path
from shlib import Cmd
import zipfile as zip

In [2]:
# Path constants
DATA_DIR = Path("rawdata")
METADATA_DIR = Path("metadata")
LOG_DIR = Path("logs")
RESULT_DIR = Path("results")

# Kaggle constants
COMPETITION_NAME = "novozymes-enzyme-stability-prediction"

In [3]:
# Initialize project directories
for d in (DATA_DIR, METADATA_DIR, LOG_DIR, RESULT_DIR):
    d.mkdir(parents=True, exist_ok=True)

In [4]:
# Download competition data
download_competition_files = Cmd(["kaggle", "competitions", "download", "-c", 
    COMPETITION_NAME, "-p", DATA_DIR])
download_competition_files.run()
dataset_file = sorted(DATA_DIR.glob(f"{COMPETITION_NAME}.*"))
dataset_file

100%|██████████| 7.06M/7.06M [00:00<00:00, 38.9MB/s]


[PosixPath('rawdata/novozymes-enzyme-stability-prediction.zip')]

In [5]:
with zip.ZipFile(dataset_file[0].resolve()) as z:
    z.extractall(path=DATA_DIR)
    dataset_file[0].unlink()

In [6]:
dataset_files = sorted(DATA_DIR.glob("*"))
dataset_files

[PosixPath('rawdata/sample_submission.csv'),
 PosixPath('rawdata/test.csv'),
 PosixPath('rawdata/train.csv'),
 PosixPath('rawdata/train_updates_20220929.csv'),
 PosixPath('rawdata/wildtype_structure_prediction_af2.pdb')]

## Data Exploration

Before we begin modelling we will have a look at the files available for the Novozyme competition to see what kind of features are available to help with our task.

In [7]:
from biopandas.pdb import PandasPdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
TRAIN_PATH = dataset_files[2]
TEST_PATH = dataset_files[1]
SAMPLE_SUBMISSION = dataset_files[0]
TRAIN_UPDATE_PATH = dataset_files[3]
TRAIN_PDB_PATH = dataset_files[4]

In [9]:
# Load available csv files
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION)
train_update_df = pd.read_csv(TRAIN_UPDATE_PATH)

In [10]:
# Use Biopythons Biopandas to load the PDB protein structure file
pdb_df = PandasPdb().read_pdb(str(TRAIN_PDB_PATH))

In [11]:
# Get it PDB file into a Python native format
protein_struct_df_dict = pdb_df.df
protein_struct_df_dict.keys()

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


In [13]:
## NOTE: tm column is melting point in Celsius (C)
train_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [14]:
train_update_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2434 entries, 0 to 2433
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            2434 non-null   int64  
 1   protein_sequence  25 non-null     object 
 2   pH                25 non-null     float64
 3   data_source       0 non-null      float64
 4   tm                25 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 95.2+ KB


In [15]:
# There were some data quality issues, need to drop NaN rows and update some pH and tm values
# See: https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/356251
train_update_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,69,NaN,NaN,NaN,NaN
1,70,NaN,NaN,NaN,NaN
2,71,NaN,NaN,NaN,NaN
3,72,NaN,NaN,NaN,NaN
4,73,NaN,NaN,NaN,NaN


In [16]:
# Extract rows which need updating in train data
bad_seq_ids = train_update_df.seq_id.values
bad_seq_ids

array([   69,    70,    71, ..., 30740, 30741, 30742])

In [17]:
# Drop those rows from the training data and append the updated rows
train_df_fix = train_df.loc[~train_df.seq_id.isin(bad_seq_ids), :]
train_df_fix = (pd.concat([train_df_fix, train_update_df])
    .sort_values(by="seq_id"))

In [18]:
train_df_fix.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [19]:
# Sanity check the non NaN columns got updated correctly
assert all(train_df_fix.iloc[bad_seq_ids].pH.dropna() == train_update_df.pH.dropna())

In [20]:
# Drop columns with NaN in the tm column, since that is our target in modelling
train_df2 = train_df_fix.loc[~train_df_fix.tm.isna(), ]

In [21]:
train_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28981 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            28981 non-null  int64  
 1   protein_sequence  28981 non-null  object 
 2   pH                28695 non-null  float64
 3   data_source       28001 non-null  object 
 4   tm                28981 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.3+ MB


In [22]:
# Due to memory constraints need to only work with shorter proteins!
train_df2 = train_df2.loc[train_df2.protein_sequence.apply(lambda x: len(x)) < 1000, :]

In [23]:
train_df2.shape

(27029, 5)

In [24]:
train_df2.protein_sequence.apply(lambda x: len(x)).max()  # Check longest sequence

999

In [25]:
# Write train/test data as a FASTA sequence file, makes it easier to make DataLoader for PyTorch
TRAIN_FASTA = RESULT_DIR / "train.fasta"
with TRAIN_FASTA.open("w", encoding="utf-8") as f:
    for i in range(train_df2.shape[0]):
        row = train_df2.iloc[i, :]
        f.write(f">{row.seq_id}|{row.pH}|{row.tm}\n")
        f.write(f"{row.protein_sequence}\n")

In [26]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   seq_id            2413 non-null   int64 
 1   protein_sequence  2413 non-null   object
 2   pH                2413 non-null   int64 
 3   data_source       2413 non-null   object
dtypes: int64(2), object(2)
memory usage: 75.5+ KB


In [27]:
# Check for duplicated proteins in test set!
assert len(test_df.protein_sequence.unique()) == test_df.shape[0]

In [28]:
TEST_FASTA = RESULT_DIR / "test.fasta"
with TEST_FASTA.open("w", encoding="utf-8") as f:
    for i in range(test_df.shape[0]):
        row = test_df.iloc[i, :]
        f.write(f">{row.seq_id}|{row.pH}|\n")
        f.write(f"{row.protein_sequence}\n")

## Modelling

Given the relatively small training set of proteins for this competition, it
is likely optimal to leverage an existing model via transfer learning to ensure
we can extract sufficient biochemical context from the protein sequences to
usefully rank the proteins by thermal stability.

Based on a brief review of the literature, I identifier the 
Evolutionary Scale Modelling (ESM) tranformer architecture from Facebook Research
as a potential candidate for transfer learning to the thermal stability task.
Pre-trained versions of their model are available via their PyPI package
`fair-esm`. The model is implemented in PyTorch and given my limited exposure
to this deep learning framework I spent a long time in trail and error
before I could succesfully export the latent space embeddings of the protein
sequences I would use for downstream modelling.

I originally hoped to simply append a dense network to the end of their model
and use that to adapt the extracted protein sequence features, as we did
in Assignment 2. However, a combination of limited compute resources (I crashed
my PC with 64 GB of RAM, RTX 3090 several times) and lack of experience with
PyTorch I decided it was best to export the protein sequence representations
in the transformer latent space and then use those as input to a dense network
in with Tensorflow/Keras for the final prediction task. 

In addition to the latent space embedded of the protein sequence, I have also 
extracted a vector of contact values which indicate the liklihood that each 
amino acid in the sequence possess any chemical bonds between them. I suspect 
this will be useful for the thermal stability task given that more interactions 
between the respective amino acids in the sequence, the greater the thermal 
energy should be required to distrupt its conformation.

In [36]:
# PyTorch
import torch
from torch import nn, optim, utils, Tensor
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

# Facebook Research Evolutionary Scale Modelling (ESM) package
from esm import FastaBatchedDataset
import esm

# Utilities
import inspect
import time
import copy

In [30]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 640, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=640, out_features=640, bias=True)
        (v_proj): Linear(in_features=640, out_features=640, bias=True)
        (q_proj): Linear(in_features=640, out_features=640, bias=True)
        (out_proj): Linear(in_features=640, out_features=640, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=640, out_features=2560, bias=True)
      (fc2): Linear(in_features=2560, out_features=640, bias=True)
      (final_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=640, out_features=640, bias=True)
        (v_proj): Linear(in_features=640, out_features=640, bias=True)
        (

In [31]:
if torch.cuda.is_available():
    model = model.cuda()

In [32]:
# Output for ESM representation of protein sequences
PROCDATA = Path("procdata")

# Model configuration
REPR_LAYERS = 30
BATCH_SIZE = 10
RETURN_CONTACTS = False
TRAIN_REPR = PROCDATA / "train_repr"
TEST_REPR = PROCDATA / "test_repr"

In [33]:
TRAIN_REPR.mkdir(parents=True, exist_ok=True)
TEST_REPR.mkdir(parents=True, exist_ok=True)

In [34]:
train_dataset = FastaBatchedDataset.from_file(TRAIN_FASTA)
train_batches = train_dataset.get_batch_indices(BATCH_SIZE, extra_toks_per_seq=1)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, collate_fn=batch_converter, batch_sampler=train_batches
)

In [35]:
with torch.no_grad():
    for batch_idx, (labels, strs, toks) in enumerate(train_dataloader):
        print(
            f"Processing {batch_idx + 1} of {len(train_batches)} batches ({toks.size(0)} sequences)"
        )
        if torch.cuda.is_available():
            toks = toks.to(device="cuda", non_blocking=True)
        
        out = model(toks, repr_layers=[REPR_LAYERS], return_contacts=RETURN_CONTACTS)
        representations = {
            layer: t.to(device="cpu") for layer, t in out["representations"].items()
        }
        if RETURN_CONTACTS:
            contacts = out["contacts"].to(device="cpu")

        for i, label in enumerate(labels):
            result = {"label": label}
            result["representations"] = representations[REPR_LAYERS][i, :]
            if RETURN_CONTACTS:
                result["contacts"] = contacts[i, :]
            output_file = TRAIN_REPR / f"{label}.pt"
            torch.save(
                result,
                output_file
            )


Processing 1 of 27029 batches (1 sequences)
Processing 2 of 27029 batches (1 sequences)
Processing 3 of 27029 batches (1 sequences)
Processing 4 of 27029 batches (1 sequences)
Processing 5 of 27029 batches (1 sequences)
Processing 6 of 27029 batches (1 sequences)
Processing 7 of 27029 batches (1 sequences)
Processing 8 of 27029 batches (1 sequences)
Processing 9 of 27029 batches (1 sequences)
Processing 10 of 27029 batches (1 sequences)
Processing 11 of 27029 batches (1 sequences)
Processing 12 of 27029 batches (1 sequences)
Processing 13 of 27029 batches (1 sequences)
Processing 14 of 27029 batches (1 sequences)
Processing 15 of 27029 batches (1 sequences)
Processing 16 of 27029 batches (1 sequences)
Processing 17 of 27029 batches (1 sequences)
Processing 18 of 27029 batches (1 sequences)
Processing 19 of 27029 batches (1 sequences)
Processing 20 of 27029 batches (1 sequences)
Processing 21 of 27029 batches (1 sequences)
Processing 22 of 27029 batches (1 sequences)
Processing 23 of 27

In [37]:
test_dataset = FastaBatchedDataset.from_file(TEST_FASTA)
test_batches = test_dataset.get_batch_indices(BATCH_SIZE, extra_toks_per_seq=1)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, collate_fn=batch_converter, batch_sampler=test_batches
)

In [38]:
with torch.no_grad():
    for batch_idx, (labels, strs, toks) in enumerate(test_dataloader):
        print(
            f"Processing {batch_idx + 1} of {len(test_batches)} batches ({toks.size(0)} sequences)"
        )
        if torch.cuda.is_available():
            toks = toks.to(device="cuda", non_blocking=True)
        
        out = model(toks, repr_layers=[REPR_LAYERS], return_contacts=RETURN_CONTACTS)
        representations = {
            layer: t.to(device="cpu") for layer, t in out["representations"].items()
        }
        if RETURN_CONTACTS:
            contacts = out["contacts"].to(device="cpu")

        for i, label in enumerate(labels):
            result = {"label": label}
            result["representations"] = representations[REPR_LAYERS][i, :]
            if RETURN_CONTACTS:
                result["contacts"] = contacts[i, :]
            output_file = TEST_REPR / f"{label}.pt"
            torch.save(
                result,
                output_file
            )

Processing 1 of 2413 batches (1 sequences)
Processing 2 of 2413 batches (1 sequences)
Processing 3 of 2413 batches (1 sequences)
Processing 4 of 2413 batches (1 sequences)
Processing 5 of 2413 batches (1 sequences)
Processing 6 of 2413 batches (1 sequences)
Processing 7 of 2413 batches (1 sequences)
Processing 8 of 2413 batches (1 sequences)
Processing 9 of 2413 batches (1 sequences)
Processing 10 of 2413 batches (1 sequences)
Processing 11 of 2413 batches (1 sequences)
Processing 12 of 2413 batches (1 sequences)
Processing 13 of 2413 batches (1 sequences)
Processing 14 of 2413 batches (1 sequences)
Processing 15 of 2413 batches (1 sequences)
Processing 16 of 2413 batches (1 sequences)
Processing 17 of 2413 batches (1 sequences)
Processing 18 of 2413 batches (1 sequences)
Processing 19 of 2413 batches (1 sequences)
Processing 20 of 2413 batches (1 sequences)
Processing 21 of 2413 batches (1 sequences)
Processing 22 of 2413 batches (1 sequences)
Processing 23 of 2413 batches (1 sequence

In [49]:
help(model)

Help on ESM2 in module esm.model.esm2 object:

class ESM2(torch.nn.modules.module.Module)
 |  ESM2(num_layers: int = 33, embed_dim: int = 1280, attention_heads: int = 20, alphabet: Union[esm.data.Alphabet, str] = 'ESM-1b', token_dropout: bool = True)
 |  
 |  Method resolution order:
 |      ESM2
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, num_layers: int = 33, embed_dim: int = 1280, attention_heads: int = 20, alphabet: Union[esm.data.Alphabet, str] = 'ESM-1b', token_dropout: bool = True)
 |      Initializes internal Module state, shared by both nn.Module and ScriptModule.
 |  
 |  forward(self, tokens, repr_layers=[], need_head_weights=False, return_contacts=False)
 |      Defines the computation performed at every call.
 |      
 |      Should be overridden by all subclasses.
 |      
 |      .. note::
 |          Although the recipe for forward pass needs to be defined within
 |          this function, one sho

In [93]:
class ESM2Regressor(nn.Module):
    """
    MultiLayerPerceptron model for regression appended to an ESM2 model for
    feature extraction for protein sequence data.
    """
    def __init__(self, esm2 = esm.pretrained.esm2_t30_150M_UR50D, 
        *args, freeze_esm = True, layers = [64, 32, 16],  **kwargs
    ):
        super(ESM2Regressor, self).__init__()
        # Initialize our pretrained ESM2 model and alphabet
        esm_model, esm_alphabet = esm2(*args, **kwargs)
        # Freeze model weights if not doing fine tuning
        if freeze_esm:
            for p in esm_model.parameters():
                p.requires_grad = False
        self.esm_alphabet = esm_alphabet
        self.feature_extractor = esm_model
        # Get the output shape from our ESM2 feature extractor
        self.input_dim = (list(self.feature_extractor.children())[-1]
            .dense.out_features)
        # Add our regression MLP, parameterizing the layers
        self.fc = nn.Sequential()
        previous_l = self.input_dim
        for l in layers:
            self.fc.append(nn.BatchNorm1d(previous_l))
            self.fc.append(nn.Linear(previous_l, l))
            self.fc.append(nn.ReLU())
            previous_l = l
        self.fc.append(
            nn.Linear(previous_l, 1)
        )

    def forward(self, input):
        features = self.feature_extractor(input)
        x = features["representations"]
        # Only using the last ESM2 layer representation for our regression
        x = x[list(x)[-1]]
        x = self.fc(x)
        return x


In [94]:
model = ESM2Regressor()
model

In [95]:
# Adapted from PyTorch docs: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def train_model(model, criterion, optimzer, schedular, tb_writer, num_epochs=100):
    """
    Function to train each epoch for our ESM2Regressor model.
    """
    since = time.time()

ESM2Regressor(
  (feature_extractor): ESM2(
    (embed_tokens): Embedding(33, 640, padding_idx=1)
    (layers): ModuleList(
      (0): TransformerLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=640, out_features=640, bias=True)
          (v_proj): Linear(in_features=640, out_features=640, bias=True)
          (q_proj): Linear(in_features=640, out_features=640, bias=True)
          (out_proj): Linear(in_features=640, out_features=640, bias=True)
          (rot_emb): RotaryEmbedding()
        )
        (self_attn_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=640, out_features=2560, bias=True)
        (fc2): Linear(in_features=2560, out_features=640, bias=True)
        (final_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=640, out_features=640, bias=True)
      